# Lookup API Integrations 

This notebook gives examples of how to use these API integrations.

## Loading Test Data

This cell loads all the initial data needed, i.e. the input IOCs list from the input txt file (see [README](README.md)).

In [ ]:
from objects import get_ioc_type

iocs:list[str] = open('data/test-input-iocs.txt', 'r').read().splitlines()
print(f'\033[90mGot \033[92m{len(iocs)} \033[90mtotal IOCs.')

# Convert the iocs to dicts with the value and type
iocs_as_dicts:list[dict] = []
for ioc in iocs: 
    iocs_as_dicts.append({
        'value': ioc,
        'type': get_ioc_type(ioc)
    })

## Loading config

This cell loads all the configuration settings from the [config JSON](config/config.json).

In [ ]:
import json 

with open('config/config.json', 'r') as file: 
    config:dict = json.load(file)               # Loading the config json
    tokens:dict[str, str] = config['tokens']    # Get the API tokens from the config dict
    
# Defining an output directory for analyzing results of testing/understanding output formats 
json_output_dir:str = 'data/test-outputs/jsons/' 
csv_output_dir:str = 'data/test-outputs/csvs/' 

## Init APIs & Conduct Lookups

This cell instantiates the API wrapper classes that interact with the lookup APIs. 

In [ ]:
from objects import IPInfoAPI, VirusTotalAPI

ipinfo_api:IPInfoAPI = IPInfoAPI(tokens['ipinfo'])
virustotal_api:VirusTotalAPI = VirusTotalAPI(tokens['virustotal'])

This cell conducts lookups on all the IOCs loaded in the first step using the APIs instantiated in the previous step.

In [ ]:
vt_results:list[dict] = virustotal_api.lookup_iocs(iocs_as_dicts)
ipinfo_results:list[dict] = ipinfo_api.lookup_iocs(iocs_as_dicts)

**Combine the VT and IPInfo results**

In [ ]:
from objects import combine_vt_ipinfo_results, SSLCert
import pandas as pd

# Combine the results
ip_results, domain_results, dns_results, sslcert_results = combine_vt_ipinfo_results(
    { ioc['value'] : ioc for ioc in vt_results },
    { ioc['value'] : ioc for ioc in ipinfo_results }
)

# Convert the SSL cert dicts to SSLCert objs, get the table rows, AND extract the alternative names from the SSL certs
sslcert_table_rows:list[SSLCert] = []
sslcert_alt_names:list[dict] = []

for d in sslcert_results: 
    this_sslcert:SSLCert = SSLCert(d)
    sslcert_table_rows.append(this_sslcert.to_table_rows())
    sslcert_alt_names.extend(this_sslcert.alt_name_to_table_rows())
    
# Create dataframes from the results
ip_results_df:pd.DataFrame = pd.DataFrame(ip_results)
domain_results_df:pd.DataFrame = pd.DataFrame(domain_results)
dns_results_df:pd.DataFrame = pd.DataFrame(dns_results)
sslcert_results_df:pd.DataFrame = pd.DataFrame(sslcert_table_rows)
sslcert_alt_names_df:pd.DataFrame = pd.DataFrame(sslcert_alt_names)



**Dump the combined results to JSONs and CSV/Excel**

In [ ]:
with open(os.path.join(json_output_dir, 'ip_results.json'), 'w+') as file: 
    json.dump(ip_results, file, indent=4)
    
with open(os.path.join(json_output_dir, 'domain_results.json'), 'w+') as file: 
    json.dump(domain_results, file, indent=4)
    
with open(os.path.join(json_output_dir, 'dns_results.json'), 'w+') as file: 
    json.dump(dns_results, file, indent=4)
    
with open(os.path.join(json_output_dir, 'sslcert_results.json'), 'w+') as file: 
    json.dump(sslcert_results, file, indent=4)

with open(os.path.join(json_output_dir, 'sslcert_alt_names.json'), 'w+') as file: 
    json.dump(sslcert_alt_names, file, indent=4)
    
ip_results_df.to_csv(os.path.join(csv_output_dir, 'ip_results.csv'), index=False)
domain_results_df.to_csv(os.path.join(csv_output_dir, 'domain_results.csv'), index=False)
dns_results_df.to_csv(os.path.join(csv_output_dir, 'dns_results.csv'), index=False)
sslcert_results_df.to_csv(os.path.join(csv_output_dir, 'sslcert_results.csv'), index=False)
sslcert_alt_names_df.to_csv(os.path.join(csv_output_dir, 'sslcert_alt_names.csv'), index=False)